# 문제 유형2

보험사에서 고객(총 29,132명) 신상 정보와 21개의 보험 상품 구매 정보를 이용하여 보험 상품 추천 시스템을 개발하고자 하며, 
제공된 데이터셋 정보는 다음과 같다.  

Train2.csv (구분자: “,”(Comma), 29,132 Rows, 26 Columns, UTF-8 Encoding)


In [1]:
import pandas as pd
import numpy as np

# !pip install mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from scipy.stats import chi2_contingency

dat=pd.read_csv('Train2.csv')
dat.head(5)

ID join_date sex marital_status  birth_year branch_code  \
0  4WKQSBB  1/2/2019   F              M        1987        1X1H   
1  CP5S02H  1/6/2019   F              M        1981        UAOD   
2  2YKDILJ  1/6/2013   M              U        1991        748L   
3  2S9E81J  1/8/2019   M              M        1990        1X1H   
4  BHDYVFT  1/8/2019   M              M        1990        748L   

  occupation_code occupation_category_code  P1  P2  ...  P12  P13  P14  P15  \
0            2A7I                     T4MS   0   0  ...    0    0    0    1   
1            2A7I                     T4MS   0   0  ...    0    0    0    1   
2            QZYX                     90QI   0   0  ...    0    0    0    0   
3            BP09                     56SI   0   0  ...    0    0    0    1   
4            NO3L                     T4MS   0   0  ...    0    0    0    0   

   P16  P17  P18  P19  P20  P21  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    1  
3    0    0    0    0    0    0  
4    0    0    1    1    0    0  

[5 rows x 29 columns]

* 다음의 전처리를 먼저 수행한 후 분석 진행하시오.


    단계 1-1. 가입 일자(join_date) 컬럼의 연도 정보를 이용하여 가입 연령(변수명: reg_age) 컬럼을 추가하시오.   
            reg_age = 가입 연도 - 출생 연도 + 1

In [2]:
step1=dat.copy()

step1['reg_age']=pd.to_datetime(step1['join_date']).dt.year - step1['birth_year'] + 1

step1[['join_date','birth_year','reg_age']].head(5)

join_date  birth_year  reg_age
0  1/2/2019        1987     33.0
1  1/6/2019        1981     39.0
2  1/6/2013        1991     23.0
3  1/8/2019        1990     30.0
4  1/8/2019        1990     30.0

    단계 1-2. 가입한 보험 상품의 수가 4개 이상(>=)인 고객은 VIP로 분류하는 VIP 컬럼을 추가하시오.  
        IF 가입한 보험 상품의 수 >= 4 then ‘VIP’ else ‘Not VIP’  (VIP는 총 1,231명)

In [3]:
step2=step1.copy()

tab1=step2.loc[:, 'P1':'P21'].sum(axis=1)
step2['VIP']=np.where(tab1 >= 4, 'VIP', 'Not VIP')

print(tab1.head(5))
print(step2['VIP'].head(5))

basetable1=step2.copy()

0    2
1    2
2    3
3    2
4    2
dtype: int64
0    Not VIP
1    Not VIP
2    Not VIP
3    Not VIP
4    Not VIP
Name: VIP, dtype: object


* 상기 전처리를 완료한 데이터 프레임(데이터 프레임명: basetable1, 212 Rows)으로 다음 분석(문제 1~3) 수행

1. (basetable1을 이용) 모집 사무소(branch_code) 별 가입 연령의 평균을 집계하고, 이 값이 가장 큰 모집 사무소를 기술하시오.  
(답안 예시) ABC


In [4]:
Q1_ans=basetable1.pivot_table(index='branch_code', values='reg_age', aggfunc='mean').idxmax().values
print('Q1 정답: ', Q1_ans)

Q1 정답:  ['49BM']


2. (basetable1을 이용) 고객들이 어떤 보험 상품들을 같이 가입했는지 연관성 분석을 다음 조건에 따라 수행하고자 한다.  

    - 21 개의 보험 상품(P1~P21 활용)을 분석 대상으로 하며, 1개의 보험상품에만 가입한 고객은 분석에서 제외
    
    - (Association Rule 모델 가이드) Min Support: 0.01, Min Confidence: 0.01, 그 외: Default

    연관성 분석 결과 중, 선행(antecedent)과 후행(consequent)이 단일 아이템으로 구성된 rule만 추출하시오. 
    이 결과를 활용하여, 보험상품 'P15'에 가입한 고객에게 추천하기 적합한 보험 상품을 Lift 기준(내림차순)으로 선정하고,
    그 때의 Lift를 기술하시오.
    
    
    - Lift는 소수점 넷째 자리 이하는 버리고 셋째 자리까지만 기술하시오.   
        (답안예시) 1.234  



In [5]:
Q2=basetable1.loc[:,'P1':'P21'].copy()

Q2_1=Q2[Q2.sum(axis=1) > 1]

apr=apriori(Q2_1, min_support=0.01, max_len=2, use_colnames=True)
Q2_out1=association_rules(apr, metric='confidence', min_threshold=0.01)

Q2_out2=Q2_out1[Q2_out1['antecedents']=={'P15'}]

Q2_out3=Q2_out2.sort_values(by='lift', ascending=False)
Q2_ans=Q2_out3['lift'].values[0]

print('Q2 정답: ', Q2_ans)  # 1.3079176923800797

Q2 정답:  1.3079176923800797


3. (basetable1을 이용) 고객의 VIP 여부가 성별(sex)과 서로 독립이라 할 수 있는지 적절한 검정을 수행하고, 검정 결과 p-value를 기술하시오.  

    - p-value는 소수점 셋째 자리 이하는 버리고 소수점 둘째 자리까지 기술  
        (답안 예시) 0.1234


In [11]:
Q3=pd.crosstab(basetable1.VIP, basetable1.sex)

Q3_out=chi2_contingency(Q3)

print(Q3_out)


Q3_ans=Q3_out[1]

print('\n Q3 정답: ', Q3_ans)

(3.985923419506938, 0.04588194490978292, 1, array([[ 8431.0209735, 19469.9790265],
       [  371.9790265,   859.0209735]]))

 Q3 정답:  0.04588194490978292
